# Days in the ICU- ML Modeling

------------
### Data Vectorization

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql.functions import *

In [4]:
dataset = spark.table("mldata_days_icu")
# display(dataset)

In [5]:
label = "days_icu"
allColumns = dataset.columns

categoricalColumns = [
  "City",
  "Marital",
  "Race",
  "Ethnicity",
  "Gender",
  "Suffix",
  "County",
  "smoker_status"
]

ignoreColumns = ["Patient", "dataset", "State", label]
numericalColumns = list(set(allColumns) - set(categoricalColumns) - set(ignoreColumns))
categoricalColumnsclassVec = [c + "classVec" for c in categoricalColumns]

stages = []

In [6]:
for categoricalColumn in categoricalColumns:
  print(categoricalColumn)
  ## Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalColumn, outputCol = categoricalColumn+"Index").setHandleInvalid("skip")
  ## Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalColumn+"Index", outputCol=categoricalColumn+"classVec")
  ## Add stages
  stages += [stringIndexer, encoder]

City
Marital
Race
Ethnicity
Gender
Suffix
County
smoker_status

In [7]:
assemblerInputs = categoricalColumnsclassVec + numericalColumns
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features").setHandleInvalid("skip")
stages += [assembler]

In [8]:
prepPipeline = Pipeline().setStages(stages)
pipelineModel = prepPipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)

In [9]:
# Note: 'TEST' is for validation while 'test' is for challenge submission
test = dataset.filter(col("dataset") == "test")
train, TEST = dataset.filter(col('dataset')=='train').randomSplit([0.7, 0.3], seed = 1337)   
# train = train.sample(False, .30, 1337)                    # Sample down for a more managable size
print("Number of rows in `train`: " + str(train.count()) + "\nNumber of rows in `test`: " + str(TEST.count()))

In [10]:
# display(dataset.select("Patient", "dataset", label, "features").filter(col("dataset") == "test"))

In [11]:
pipelineModel.write().overwrite().save("/mnt/data/ml/daysicu/pipeline")
# display(dbutils.fs.ls("/mnt/data/ml/daysicu/pipeline"))

### Linear Regression

In [13]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [14]:
# Using 'lm' for linear model and to avoid confusion with 'lr' for logistic regression

# Load training data
lm = LinearRegression(featuresCol = 'features', labelCol=label)

# Fit the model
lmModel = lm.fit(train)

# Summarize the model over the training set and print out some metrics
trainingSummary = lmModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
trainingSummary.residuals.show()

Coefficients: [-0.05661695786975474,0.03514625897016836,0.4194494046868216,0.2118691325582882,-1.42133736161548,0.11733950028684996,0.49234065914811836,-0.12221450862707035,0.10558733293306792,0.17128131520248754,-0.2355545380306913,-0.11577969997317818,0.001353209265643388,0.7371263116989601,0.504557323923116,-0.8335232089445329,0.048282248764671694,0.7874561165469361,0.026645870297322325,-0.3460545761432003,0.3742215755032091,-0.254235159111022,0.12798984541504427,1.528063566939619,0.6734122281267249,-0.14791243123208242,-0.6805827824852005,-0.23042970144530223,-0.8708031557493194,1.4101151930215319,0.4224770201836111,-0.08081473240557412,0.6911762117720218,0.8899593487326676,-0.40346343985099004,-0.07807423909445421,-1.022697685298238,0.2982178254408924,-0.054687903452170145,-1.123040926539549,-0.7699686425015105,-0.5099630191293602,-1.4563123858784663,0.2026889767645624,0.6888109736378946,0.2684605415516148,0.04386411218119233,0.3746519193865231,1.071840233260398,-0.2528640352252734,-0.340333288193612,0.5143840502872266,-1.8715030048284873,0.22254049251939073,0.2915164170008125,0.18960004428550595,0.4228080640523571,0.3157425009778285,0.23174892596151314,0.9200275731979726,-0.5807582097902425,-0.08209208899337798,0.5181013515524672,0.03868774971015615,-0.31432337818347195,-1.0327856947929315,-0.045921529608180925,0.5753977041138927,-0.5485478298936405,-0.6021605310549087,-0.31881679941725766,0.9020342793025921,0.4079948315654205,-2.0488208917571877,-0.5469045992765944,1.8202096938427879,-0.8651799009215912,0.2429863623461056,0.3947840842870087,-0.26391154810399275,-0.06224251284071834,-0.6959867645029878,0.09212281930801536,-1.3759603292222304,0.20032353014278345,1.0652811765032064,0.22558274027691047,-0.6595861418837252,-0.12986253845386217,-1.0215826707160376,-0.4546192383444725,-1.2454927439388042,1.0322581425134294,-0.9933206554477635,0.4271454676938928,1.1879517583639343,-0.08013364808582395,0.2249016033266156,-0.37199403359659694,-0.8974995144942719,-0.41201118823682703,-0.7701827425509087,-0.9985203456853946,-0.7190418733065481,-1.0650109530529899,0.9594603965283555,-0.1222618308353285,-0.16478010706076024,1.0241758229681865,-0.7208281453323099,-0.06412634324431657,1.443306643197489,0.7639014923681116,-0.6768238047707232,-0.03512244578691934,-0.41398452624632787,0.773115399995868,-0.0952109374150965,-0.8715313047072529,1.0486707647033047,1.1112699676617883,1.948606298384643,-0.06784042730729241,-0.641238437695022,-0.230614082070034,0.7784494620984579,-0.4129713823622282,-0.9616370985324715,1.286854068495975,1.0709206606118968,-0.7540423022249515,-1.204495800069729,-1.367850278637735,-1.028660455721925,2.4878707286468065,0.5790700995566354,-1.1119205168867745,-0.7643901731231458,3.297194264798689,-1.5014606131136945,-0.7462404401751415,-2.7970103743633277,0.9601022894047216,0.0271310088220086,0.1968166112976085,0.18198480201744274,2.278778145640764,-0.6606643452596476,0.4758681863058351,-1.0253866658425232,-1.320032832742085,1.9712714861826939,-1.07166861372259,0.41125504554480746,1.0931223178667049,0.3642923652828626,-0.12066326456442045,0.1851599399557058,0.16366140692132283,0.20693726487536232,0.945367651099707,-0.23107554260031188,-0.7517309652380234,-0.02791117412307922,-0.6713103586976176,0.18544343063898608,1.5351734815577782,0.05391936655220765,1.764191491646425,2.836170583650779,0.027186885376120233,1.0315503167901643,-1.4707246475539397,-1.5399631783087584,-0.8900563356991943,1.9207272818854595,-1.8397948793474255,0.33342361098442963,1.647315760848991,0.4822085159174829,-1.185872690339074,3.559281180492598,0.4028249581816674,0.7340777028689166,1.1220717755900698,-3.800918986770306,-0.7786400771749155,0.7918678771509261,-1.4689675261493091,-0.07803841526819957,1.4981867987999293,-0.8412373770751783,-0.16158314713692165,3.089026770135453,0.2602156741637135,-3.0984816168197287,-0.27043002042855,-2.637219266844185,2.4420805441242943,1.3747781029747188,0.10290348585700491,-1.9658680894669285,-1.641876774405203,-

In [15]:
# Create ParamGrid for Cross Validation
lmParamGrid = (ParamGridBuilder()
             .addGrid(lm.regParam, [0.01, 0.1, 0.5])
             .addGrid(lm.elasticNetParam, [0.0, 0.1, 0.25])
             .addGrid(lm.maxIter, [1, 5, 10])
             .build())

# Evaluate model
lmEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol=label, metricName="rmse")

In [16]:
# Create 5-fold CrossValidator
lmCv = CrossValidator(estimator = lm,
                    estimatorParamMaps = lmParamGrid,
                    evaluator = lmEvaluator,
                    numFolds = 5)

# Run cross validations
lmCvModel = lmCv.fit(train)
lmCvModel.write().overwrite().save("/mnt/data/ml/daysicu/lm")
# print(lmCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
/databricks/spark/python/pyspark/ml/util.py:760: UserWarning: Logging to MLflow failed. Tuning will run without MLflow logging. Caught MlflowException with message: API request to https://eastus2.azuredatabricks.net/api/2.0/mlflow/runs/create failed to return code 200 after 3 tries
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/ml/util.py", line 752, in _func_call_catch_exception
 func(**kwargs)
 File "/databricks/spark/python/pyspark/ml/util.py", line 876, in _log_cv
 _MLflowInstrumentation._log_tuning_runs(run_uuid, fit_uuid, est, epm, metric_names, metrics)
 File "/databricks/spark/python/pyspark/ml/util.py", line 960, in _log_tuning_runs
 experiment_id=exp_id_for_run, tags=tags)
 File "/databricks/python/lib/python3.7/site-packages/mlflow/tracking/client.py", line 123, in create_run
 return self._tracking_client.create_run(experiment_id, start_time, tags)
 File "/databricks/python/lib/python3.7/site-packages/mlflow/tracking/_tracking_service/client.py", line 88, in create_run
 tags=[RunTag(key, value) for (key, value) in iteritems(tags)]
 File "/databricks/python/lib/python3.7/site-packages/mlflow/store/tracking/rest_store.py", line 117, in create_run
 response_proto = self._call_endpoint(CreateRun, req_body)
 File "/databricks/python/lib/python3.7/site-packages/mlflow/store/tracking/rest_store.py", line 32, in _call_endpoint
 return call_endpoint(self.get_host_creds(), endpoint, method, json_body, response_proto)
 File "/databricks/python/lib/python3.7/site-packages/mlflow/utils/rest_utils.py", line 136, in call_endpoint
 host_creds=host_creds, endpoint=endpoint, method=method, json=json_body)
 File "/databricks/python/lib/python3.7/site-packages/mlflow/utils/rest_utils.py", line 80, in http_request
 (url, retries))
mlflow.exceptions.MlflowException: API request to https://eastus2.azuredatabricks.net/api/2.0/mlflow/runs/create failed to return code 200 after 3 tries

 stack=exc_str.getvalue()))
CrossValidatorModel_fe1821e77ff2

In [17]:
# List important features
import pandas as pd

featurelist = pd.DataFrame(dataset.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+dataset.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")
featurelist["Coefficient"] = pd.DataFrame(lmCvModel.bestModel.coefficients.toArray())
featurelist = sqlContext.createDataFrame(featurelist)
# display(featurelist)

idx,name,Coefficient
0,CityclassVec_Boston,0.0
1,CityclassVec_Worcester,0.0
2,CityclassVec_Springfield,0.0
3,CityclassVec_Lowell,0.0
4,CityclassVec_Cambridge,0.0
5,CityclassVec_New Bedford,0.0
6,CityclassVec_Quincy,0.0
7,CityclassVec_Brockton,0.0
8,CityclassVec_Newton,0.0
9,CityclassVec_Lynn,0.0


In [18]:
# Get Model Summary Statistics
lmCvSummary = lmCvModel.bestModel.summary
# print("Coefficient Standard Errors: " + str(lmCvSummary.coefficientStandardErrors))  # No s.e. avaialble for this model
# print("P Values: " + str(lmCvSummary.pValues)) # Last element is the intercept       # No p-values available either

# Use test set here so we can measure the accuracy of our model on new data
lmPredictions = lmCvModel.transform(TEST)
lmPredictions_test = lmCvModel.transform(test)   # (for submission)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', lmEvaluator.evaluate(lmPredictions))

RMSE: 2.696249726777729

### Poisson Regression
Poisson is a family within Generalized Linear Models for count dependent variables.

In [20]:
from pyspark.ml.regression import GeneralizedLinearRegression

In [21]:
glm = GeneralizedLinearRegression(family="poisson", featuresCol = 'features', labelCol=label)   # No improvementwith a sqrt link function either

# Fit the model
glmModel = glm.fit(train)

Coefficients: [-0.01575733734563751,0.0019475984470586926,0.06097087137206142,0.04325315741381155,-0.17146336765367753,0.007849009443225,0.05689453367838236,0.013100953745424835,-0.07923505815264173,0.028206709704745397,-0.07892196190758752,-0.05206361460612364,0.035303175702307926,0.1632046879186884,0.004687636305968898,-0.13342156798434085,0.0448501394892391,0.018789362457634778,0.0006183926492413423,0.04371984823093626,-0.036307604031976755,-0.07589453762153386,-0.03239391307071679,0.2474511928245448,-0.08781226783359418,0.08722787281569734,0.07838961055353234,0.05529006779657053,-0.13444849699247272,0.13407211379560127,0.02997635273007142,0.008900476016792565,0.0006810220118458988,0.03510890366517646,-0.06422420996505819,-0.039985539649670845,-0.19487761441298784,0.015672545475177622,-0.13119603737026553,-0.41751388703850667,-0.15071088805918756,-0.2808911360256395,-0.07668233860958322,0.06691129638664772,-0.06844559925425063,0.11917826664893363,-0.03436438061303976,-0.056359474044980866,0.025217505275520125,0.020465597103489122,0.18177024669574313,0.19062611320046832,-0.24690126479294563,0.14747719338224693,0.07573826583069537,0.0937956686291468,-0.04972176164849753,-0.008254288877252025,0.07523341263651297,0.1884867235889904,-0.12640611433353974,-0.009389557327036828,-0.1157968584267259,-0.10201375262132492,0.04279024933058708,-0.16284784791477086,0.10164969656347612,0.13051311638256227,-0.06439137204539677,-0.10008843616272384,0.10001122306781976,0.12241154784177864,-0.11771037693365198,-0.24883021434720032,-0.029128491942883073,0.16748433829762385,-0.07336079872989862,-0.011206234896180632,0.08849303717372736,-0.16476688496058606,0.037884270917928606,-0.06945966704982805,-0.07043704296086355,-0.02033475786739378,-0.1352823748010813,0.21343907525931002,0.051185060371221676,-0.13499393867718665,-0.004100912693693531,-0.2695183516826156,0.1663151567755329,-0.10629688268899229,0.06463888417947608,-0.07666338458568987,0.03248865758565086,0.08619167036621043,0.06309681376711454,-0.2259927516939603,0.05052305531611805,0.0021775216509755716,-0.10117917225241058,-0.07665514075153726,0.09598728934513859,0.021505464182555042,-0.29329285862839183,-0.09677015679356445,-0.27046919505057393,0.09382684046686413,0.10110889527050917,-0.10748862193338311,-0.062168922617864296,0.053836050190559266,0.13859769086882828,-0.09372480990357412,0.031375895367409895,-0.0023315436619035404,0.12092673398278588,0.1926543887359353,-0.3380007855428394,0.15740799417484988,-0.07600715203599241,0.18949423801651571,-0.027037126092333327,-0.10746945269215846,0.024232017998368487,0.273615194713907,-0.14089856048487986,-0.3013809583763315,0.1853006515209394,0.17347586731078335,-0.18689059554316748,-0.13923522475168587,-0.14853823315223694,-0.1596688944693554,0.34185926365582475,0.001994564473292939,-0.30914256852913796,-0.1757081492914533,-0.23280984181485434,-0.07601908602942707,-0.0017003485190897273,-0.522432813513163,-0.006978017362867369,0.024079408869970766,0.04255791876757523,-0.2474623855864296,0.20692581200054463,-0.20115927637770986,-0.38207239961693235,-0.2849397838995056,-0.2368601471918308,0.17805708759387978,-0.10442814953601162,0.08893954470470769,0.21897298005539173,0.08036098689760772,0.14549901671196838,-0.14187264618386947,-0.49296616282174366,0.2572843178123064,-0.05637582154975628,0.07256026226167023,-0.15505681747391292,-0.1001404832574883,0.35189402536948095,-0.008694005573645336,0.07849301390939291,0.04403838637421573,0.26653823191179254,0.14988730194058664,-0.2530185353779937,0.05978824278984254,-0.2628180650579685,-0.10492594870989,-0.33412374191989735,-0.45420568901969677,-0.2737858404902272,0.38599856569195495,0.37214583100160736,-0.10089157250576548,-0.14169235788053303,0.2333346336031627,0.1399143355349666,-0.009207576474739482,0.016283107003085297,-0.9577495764702406,-0.140783723271976,0.2734195708811885,-0.14032071837680277,0.016819950105972405,0.16548326817491063,0.04904681099895334,-0.3929218501774258,0.12062835123572672,0.2

In [22]:
# Summarize the model over the training set and print out some metrics
summary = glmModel.summary
# print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
# print("T Values: " + str(summary.tValues))
# print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
# print("Deviance Residuals: ")
summary.residuals().show()

Dispersion: 1.0
Null Deviance: 4822.712815262465
Residual Degree Of Freedom Null: 3549
Deviance: 3579.7044319795773
Residual Degree Of Freedom: 2362
AIC: 18624.807365569468
Deviance Residuals: 
+--------------------+
 devianceResiduals|
+--------------------+
 0.9120442665638192|
 -0.3125754476937581|
 1.2361618835257273|
 1.1882198487293827|
 0.4197598146709802|
 1.2081557886646308|
 -0.1307929886207205|
 -0.5241953657856028|
 0.5547535497474029|
 -1.188540008977228|
-0.01213177882742...|
 0.231220787059446|
 0.2875245756099323|
 0.6033706969265106|
 0.7205922045551334|
 0.6760621475004408|
 0.13708594685808656|
 -1.1299974854860926|
 -1.7459705933505791|
 0.6091675105180792|
+--------------------+
only showing top 20 rows

In [23]:
# Create ParamGrid for Cross Validation
glmParamGrid = (ParamGridBuilder()
             .addGrid(glm.regParam, [0.01, 0.1, 0.5])
             .addGrid(glm.maxIter, [1, 5, 10])
             .build())

# Evaluate model
glmEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol=label, metricName="rmse")

# Create 5-fold CrossValidator
glmCv = CrossValidator(estimator = glm,
                    estimatorParamMaps = glmParamGrid,
                    evaluator = glmEvaluator,
                    numFolds = 5)

# Run cross validations
glmCvModel = glmCv.fit(train)
glmCvModel.write().overwrite().save("/mnt/data/ml/daysicu/glm")
print(glmCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_14cd0b821909

In [24]:
# List important features
featurelist = pd.DataFrame(dataset.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+dataset.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")
featurelist["Coefficient"] = pd.DataFrame(glmCvModel.bestModel.coefficients.toArray())
featurelist = sqlContext.createDataFrame(featurelist)

# display(featurelist)

idx,name,Coefficient
0,CityclassVec_Boston,-1.0113432571582322E-8
1,CityclassVec_Worcester,-2.4946547767590415E-8
2,CityclassVec_Springfield,1.0800614222951319E-7
3,CityclassVec_Lowell,1.905353181966558E-8
4,CityclassVec_Cambridge,-2.3298119701259441E-7
5,CityclassVec_New Bedford,-5.0157399347679756E-8
6,CityclassVec_Quincy,4.268054716366873E-8
7,CityclassVec_Brockton,-2.3796196857030758E-8
8,CityclassVec_Newton,-1.0697340728064965E-7
9,CityclassVec_Lynn,2.8971629124817267E-8


In [25]:
# Get Model Summary Statistics
glmCvSummary = glmCvModel.bestModel.summary
# print("Coefficient Standard Errors: " + str(glmCvSummary.coefficientStandardErrors))
# print("P Values: " + str(glmCvSummary.pValues)) # Last element is the intercept

# Use test set here so we can measure the accuracy of our model on new data
glmPredictions = glmCvModel.transform(TEST)
# glmpredictions_test = glmcvModel.transform(test)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', glmEvaluator.evaluate(glmPredictions))

RMSE: 2.7576984789039276

### Decision Tree Regression

In [27]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
# Load training data
dtr = DecisionTreeRegressor(labelCol=label, featuresCol="features")

In [29]:
# Create ParamGrid for Cross Validation
dtrParamGrid = (ParamGridBuilder()
             .addGrid(dtr.maxDepth, [2, 5, 10])
             .addGrid(dtr.maxBins, [10, 25, 50, 100])
             .build())

In [30]:
# Evaluate model
dtrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")

In [31]:
# Create 5-fold CrossValidator
dtrCv = CrossValidator(estimator = dtr,
                      estimatorParamMaps = dtrParamGrid,
                      evaluator = dtrEvaluator,
                      numFolds = 5)

In [32]:
# Fit the model
dtrCvModel = dtrCv.fit(train)
dtrCvModel.write().overwrite().save("/mnt/data/ml/daysicu/dtr")
print(dtrCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_5e88b4019787

In [33]:
# Use test set here so we can measure the accuracy of our model on new data
dtrPredictions = dtrCvModel.transform(TEST)
dtrPredictions_test = dtrCvModel.transform(test)   # (for submission)

In [34]:
print('RMSE:', dtrEvaluator.evaluate(dtrPredictions))

RMSE: 2.7116213946589727

### Random Forest Regression

In [36]:
from pyspark.ml.regression import RandomForestRegressor

In [37]:
# Load training data
rfr = RandomForestRegressor(labelCol=label, featuresCol="features")

In [38]:
# Create ParamGrid for Cross Validation
rfrParamGrid = (ParamGridBuilder()
             .addGrid(rfr.maxDepth, [2, 5, 10])
             .addGrid(rfr.maxBins, [10, 25, 50, 100])
             .addGrid(rfr.numTrees, [5, 20, 50])
             .build())

In [39]:
# Evaluate model
rfrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")

In [40]:
# Create 5-fold CrossValidator
rfrCv = CrossValidator(estimator = rfr,
                      estimatorParamMaps = rfrParamGrid,
                      evaluator = rfrEvaluator,
                      numFolds = 5)

In [41]:
# Fit the model
rfrCvModel = rfrCv.fit(train)
rfrCvModel.write().overwrite().save("/mnt/data/ml/daysicu/rfr")
print(rfrCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_fc2125677895

In [42]:
# Use test set here so we can measure the accuracy of our model on new data
rfrPredictions = rfrCvModel.transform(TEST)
rfrPredictions_test = rfrCvModel.transform(test)    # (for submission)

In [43]:
print('RMSE:', rfrEvaluator.evaluate(rfrPredictions))

RMSE: 2.692680321390783

In [44]:
import pandas as pd
from pyspark.ml.tuning import CrossValidatorModel

rfrCvModel = CrossValidatorModel.load("/mnt/data/ml/daysicu/rfr")

def ExtractFeatureImportance(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
  
# ExtractFeatureImportance(model.stages[-1].featureImportances, dataset, "features")
dataset_fi = ExtractFeatureImportance(rfrCvModel.bestModel.featureImportances, train, "features")
dataset_fi = sqlContext.createDataFrame(dataset_fi)
display(dataset_fi)

idx,name,score
1167,age_years,0.13916857645347314
928,QALY_Avg,0.05913856499946424
881,QALY_Max,0.05404866154087369
462,QALY_Min,0.03568716143415229
811,29463-7_Avg,0.02228119897031035
1098,DALY_Min,0.013976082647022306
1175,8302-2_Max,0.01356648612713769
861,29463-7_Max,0.013309308090944732
1104,8462-4_Min,0.012645823350196948
539,DALY_Max,0.012179870601267183


In [45]:
rfrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")
rfrEvaluator.evaluate(rfrPredictions)

Out[26]: 2.692680321390783